In [2]:
import polars as pl
import numpy as np
import os
import matplotlib.pyplot as plt
import emd
from dtw import *
from utils import *

time = []
fs = []
hr = []
mx = []

# Recordings files paths
path = os.getcwd() + r"\data_healthy"
file_paths = sorted([os.path.join(path, file) for file in os.listdir(path)], key=extract_recording_number)

# Read data from the files
for file_path in file_paths:
    df = pl.read_csv(file_path, separator=";", decimal_comma=True)
    t_hat, _ = convert_datetime_to_time(df["DateTime"].to_numpy())
    time.append(t_hat)
    hr.append(df["hr"].to_numpy())
    if df["FVL"].mean() > df["FVR"].mean():
        mx.append(df["mx_l"].to_numpy())
    else:
        mx.append(df["mx_r"].to_numpy())

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [3]:
dtw_distances = []
dtw_normalized_distances = []
drop_dtw_distances = []
drop_dtw_normalized_distances = []

for record_number in range(len(mx)):
    imf_mx = emd.sift.sift(mx[record_number])
    n = imf_mx.shape[1]
    imf_mx = imf_mx[:, n - 3] + imf_mx[:, n - 2] + imf_mx[:, n - 1]

    imf_hr = emd.sift.sift(hr[record_number])
    n = imf_hr.shape[1]
    imf_hr = np.sum(imf_hr[:, 1:], axis=1)

    imf_hr = (imf_hr - np.min(imf_hr)) / (np.max(imf_hr) - np.min(imf_hr))
    imf_mx = (imf_mx - np.min(imf_mx)) / (np.max(imf_mx) - np.min(imf_mx))
    
    # DTW
    alignment = dtw(imf_hr, imf_mx, keep_internals=False, distance_only=False)
    dtw_distances.append(alignment.distance)
    normalized_distance = alignment.distance / (len(imf_hr) + len(imf_mx))
    dtw_normalized_distances.append(normalized_distance)

    # DropDTW
    zx_costs, x_drop_costs, z_drop_costs = compute_all_costs(
        series1=imf_hr, 
        series2=imf_mx, 
        drop_cost_type="percentile", 
        percentile=75
    )

    min_cost, matched_indices, dropped1, dropped2 = double_drop_dtw(
        costs=zx_costs,
        drop_costs1=x_drop_costs,
        drop_costs2=z_drop_costs,
        contiguous=True
    )

    drop_dtw_distances.append(min_cost)
    normalized_drop_distance = min_cost / (len(imf_hr) + len(imf_mx))
    drop_dtw_normalized_distances.append(normalized_drop_distance)

print("DTW Distances:", dtw_distances)
print("DTW Normalized Distances:", dtw_normalized_distances)
print("DropDTW Distances:", drop_dtw_distances)
print("DropDTW Normalized Distances:", drop_dtw_normalized_distances)

DTW Distances: [32.59007338916259, 43.35723890233348, 83.70371223351452, 29.99836236457856, 29.464892557361274, 30.061725734072684, 31.791097429315016, 35.109785618710625, 32.68558866431215, 40.84897723948792, 49.84877722287814, 105.09345870492436, 34.99147050589447, 32.59877541659738, 61.65723490726405, 12.560307599003394, 14.575346929216435, 41.25216480238002, 14.925892278257592, 55.810604836870674, 53.36219730997616, 44.187854897154196, 50.437261450571235, 47.06365869891019, 37.96390594346855, 35.95919471018399, 36.671262340401704, 36.80628771645139, 35.13601832714768, 36.99927129562863, 42.249213093163895, 34.96062574921634, 17.560897291542055, 40.470349163367345]
DTW Normalized Distances: [0.08531432824388113, 0.07714811192586028, 0.1453189448498516, 0.07352539795239843, 0.06490064439947417, 0.07368070032860952, 0.05998320269682079, 0.12106822627141595, 0.06408938953786696, 0.07141429587323063, 0.08335915923558218, 0.2101869174098487, 0.05662050243672244, 0.0665281130950967, 0.096

In [5]:
import pandas as pd

# Create a DataFrame with the distances
distances_df = pd.DataFrame({
    'DTW Distances': dtw_distances,
    'DTW Normalized Distances': dtw_normalized_distances,
    'DropDTW Distances': drop_dtw_distances,
    'DropDTW Normalized Distances': drop_dtw_normalized_distances
})

# Save the DataFrame to a CSV file
distances_df.to_csv(os.path.join(os.getcwd(), 'distances.csv'), index=False)